In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np

from stockMarket.core.contract import Contract
from stockMarket.api import populate_contracts

contract = Contract(ticker='AAPL')
contracts = populate_contracts([contract])
print(contracts[0].equity_ratio)

[17.62592071 14.36464402 17.97425656 20.17333152 26.73078968 29.2971495 ]


### Retrieve Data

In [2]:
from stockMarket.core import get_tickers_from_index
from stockMarket.api import populate_contracts

tickers = get_tickers_from_index('sp500')
contracts = populate_contracts(tickers)

### Perform Screening

In [44]:
import numpy as np
import operator

from stockMarket.core import Ranking, RangeRankingObject
from stockMarket.core.ranking import ValueRankingConstraint

equity_ratio_ranking = RangeRankingObject("Equity Ratio", lambda x: x.equity_ratio, [10, 30], [0,1,2])
netto_margin_ranking = RangeRankingObject("Netto Margin", lambda x: x.netto_margin, [10, 20], [0,1,2])
return_on_assets_ranking = RangeRankingObject("Return on Assets", lambda x: x.return_on_assets, [5, 10], [0,1,2])
goodwill_ranking = RangeRankingObject("Goodwill", lambda x: x.goodwill_ratio, [0, 30], [0,1,0])
gearing_ranking = RangeRankingObject("Gearing", lambda x: x.gearing, [20, 60], [2,1,0])

years_back = 1

trailing_pe_constraint = ValueRankingConstraint(lambda x: x.price_to_earnings(years_back=years_back), operator.gt, 0)
peg_trailing_3y_ranking = RangeRankingObject(
    "PEG Trailing 3Y",
    lambda x: x.peg_trailing_3y(years_back=years_back),
    [0.8, 1.2], [2,1,0],
    constraints=[trailing_pe_constraint]
)

trailing_pr_constraint = ValueRankingConstraint(lambda x: x.price_to_revenue(years_back=years_back), operator.gt, 0)
prg_trailing_3y_ranking = RangeRankingObject(
    "PRG Trailing 3Y",
    lambda x: x.prg_trailing_3y(years_back=years_back),
    [0.8, 1.2], [2,1,0],
    constraints=[trailing_pr_constraint]
)

trailing_pfc_constraint = ValueRankingConstraint(lambda x: x.price_to_free_cashflow(years_back=years_back), operator.gt, 0)
pfcg_trailing_3y_ranking = RangeRankingObject(
    "PFCG Trailing 3Y",
    lambda x: x.pfcg_trailing_3y(years_back=years_back),
    [0.8, 1.2], [2,1,0],
    constraints=[trailing_pfc_constraint]
)



ranking_list = [
    equity_ratio_ranking,
    netto_margin_ranking,
    return_on_assets_ranking,
    goodwill_ranking,
    gearing_ranking,
    peg_trailing_3y_ranking,
    prg_trailing_3y_ranking,
    pfcg_trailing_3y_ranking,
]



ranking = Ranking(contracts, ranking_list, years_back=years_back)
ranking.rank()
ranking.ranking.to_csv("ranking_contracts.csv", sep='\t', encoding='utf-8', float_format='%.2f')
ranking.ranking

,Name,Sector,Relative Score,Absolute Score,No Data/Constraints/Tot.,Equity Ratio,Equity Ratio Score,Netto Margin,Netto Margin Score,Return on Assets,...,Goodwill,Goodwill Score,Gearing,Gearing Score,PEG Trailing 3Y,PEG Trailing 3Y Score,PRG Trailing 3Y,PRG Trailing 3Y Score,PFCG Trailing 3Y,PFCG Trailing 3Y Score
ABNB,"Airbnb, Inc.",Consumer Cyclical,100.00%,15/15,0/0/8,34.667664,2,22.538397,2,11.803217,...,11.690647,1,-111.097122,2,-0.187126,2,0.568459,2,0.221891,2
TROW,"T. Rowe Price Group, Inc.",Financial Services,100.00%,15/15,0/0/8,75.919198,2,24.010542,2,13.380227,...,29.897619,1,-12.387578,2,-2.015279,2,0.622631,2,0.625785,2
MRNA,"Moderna, Inc.",Healthcare,100.00%,15/15,0/0/8,57.339171,2,66.060311,2,49.462889,...,0.000000,1,-59.703075,2,-0.016902,2,0.016155,2,-0.013178,2
VRTX,Vertex Pharmaceuticals Incorporated,Healthcare,100.00%,15/15,0/0/8,76.650194,2,37.197532,2,18.302123,...,7.820193,1,-59.333559,2,0.544946,2,0.288396,2,0.452476,2
TER,"Teradyne, Inc.",Technology,100.00%,15/15,0/0/8,67.305591,2,27.399989,2,26.633652,...,16.615886,1,-33.548665,2,0.677408,2,0.263381,2,0.616589,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EQIX,"Equinix, Inc.",Real Estate,13.33%,2/15,0/0/8,37.960028,2,9.697574,0,2.323747,...,49.141611,0,124.759816,0,11.046766,0,1.440310,0,2.171030,0
PANW,"Palo Alto Networks, Inc.",Technology,13.33%,2/15,0/0/8,1.713782,0,-4.853222,0,-2.178951,...,1308.428571,0,429.809524,0,0.000000,0,0.556199,2,1.493114,0
AES,The AES Corporation,Utilities,13.33%,2/15,0/0/8,8.488305,0,-3.671125,0,-1.240785,...,42.065761,0,682.344532,0,0.000000,0,2.067031,0,-1.516856,2
SYY,Sysco Corporation,Consumer Defensive,0.00%,0/9,3/0/8,5.120202,0,0.407377,0,0.952238,...,322.149760,0,872.673447,0,NaN,0,NaN,0,NaN,0


### Write Email

In [45]:
from stockMarket.utils import write_email

emails = ["benjaminlantschner@gmail.com", "97gamjak@gmail.com"]
subject = "Stock Market Ranking"
body = ""
attachment = "ranking_contracts.csv"
write_email(emails, subject, body, attachment)